In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import os
import time
import pandas as pd

In [2]:
pd.read_csv('equipment_data.csv').to_excel('equipment_data.xlsx')

In [3]:
service = Service("C://Users//servi//inter-scraping//chromedriver-win64//chromedriver-win64//chromedriver.exe")
chrome_profile_path = r'C:\Users\servi\AppData\Local\Google\Chrome\User Data\Default'

# Set up Chrome options
# Configuração das opções do Chrome
options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir={chrome_profile_path}")
options.add_experimental_option("detach", True)  # Isso permite que o Chrome permaneça aberto após o script terminar

# Inicializa o Chrome com as opções
driver = webdriver.Chrome(service=service)
"""
https://gestao.granitopagamentos.com.br/Login/Index
sandro.leao@creditoessencial.com.br
11S@ndr01003
"""

'\nhttps://gestao.granitopagamentos.com.br/Login/Index\nsandro.leao@creditoessencial.com.br\n11S@ndr01003\n'

In [5]:
linhas = driver.find_elements(By.XPATH, "//table[@id='tabelaClientes']/tbody/tr")


In [7]:
linhas[0].click()

In [21]:
def change_window(window_name):
    link_pesquisa = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.LINK_TEXT, window_name))
    )
    try:
        link_pesquisa.click()
    except ElementClickInterceptedException:
        time.sleep(0.5)
    wait_for_processing_to_disappear(driver)

def wait_for_processing_to_disappear(driver, timeout=10):
    """ Aguarda a aba 'Processando...' desaparecer antes de continuar """
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.XPATH, "//*[contains(text(), 'Processando')]"))
        )
        print("Aba 'Processando...' desapareceu. Continuando...")
    except Exception as e:
        print(f"Erro ou tempo esgotado ao esperar a aba 'Processando...': {e}")

def get_element_text_by_xpath(xpath, option = 'value'):
    element_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )

    # Capturar o valor do input
    element_text = element_input.get_attribute(option)
    return element_text


In [19]:
element_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="RamoAtividadePF"]/option'))
            )
element_input.get_attribute('text')

'Venda porta-a-porta e venda direta'

In [57]:
equipment_data = {
    'name': [], 'cpf_cnpj': [],
    'equipment_id': [], 'pdv': [], 'phone': [], 'email': [],
    'fantase_name': [], 'branch_of_activity': [], 'cep': [],
    'steet_name': [], 'address_number': [], 'address_complement': [], 'neighborhood_name': [],
    'city_name': [], 'state_name': []
}

In [59]:
fantasy_name

'Victor Emannuel Medeiros Silva'

In [61]:
page = 1
while True:
    client_rows = driver.find_elements(By.XPATH, "//table[@id='tabelaClientes']/tbody/tr")
    for i, client_row in enumerate(client_rows):
        try:
            cpf_cnpj = client_row.find_element(By.XPATH, ".//td[3]").text
            name = client_row.find_element(By.XPATH, ".//td[4]").text
            print(f'{i+1}/{len(client_rows)} da pag. {page}')
            
            client_row.click()
            phone = get_element_text_by_xpath('//*[@id="tabelaTelefones"]/tbody/tr/td[2]/input')
            email = get_element_text_by_xpath('//*[@id="tabelaEmails"]/tbody/tr/td[2]/input')
            fantasy_name = get_element_text_by_xpath('//*[@id="Fantasia"]')
            branch_of_activity = get_element_text_by_xpath('//*[@id="RamoAtividadePF"]/option', 'text')
            cep = get_element_text_by_xpath('//*[@id="CEP"]')
            street_name = get_element_text_by_xpath('//*[@id="Endereco"]')
            address_number = get_element_text_by_xpath('//*[@id="Numero"]')
            address_complement = get_element_text_by_xpath('//*[@id="Complemento"]')
            neighborhood_name = get_element_text_by_xpath('//*[@id="Bairro"]')
            city_name = get_element_text_by_xpath('//*[@id="Cidade"]/option', 'text')
            state_name = get_element_text_by_xpath('//*[@id="UF"]/option', 'text')
            
            time.sleep(1)
            change_window('Equipamentos')
            time.sleep(1)

            # Espera até que os elementos estejam disponíveis
            max_retries = 5
            retries = 0
            while retries < max_retries:
                try:
                    equipment_rows = driver.find_elements(By.XPATH, "//table[@id='tabelaPDV']/tbody/tr")
                    
                    if not equipment_rows:
                        break  # Nenhum equipamento encontrado, segue para o próximo cliente

                    for equipment_row in equipment_rows:
                        equipment_id = equipment_row.find_element(By.XPATH, ".//td[2]").text
                        pdv = equipment_row.find_element(By.XPATH, ".//td[1]").text
                        equipment_data['name'].append(name)
                        equipment_data['cpf_cnpj'].append(cpf_cnpj)
                        equipment_data['equipment_id'].append(equipment_id)
                        equipment_data['pdv'].append(pdv)
                        equipment_data['phone'].append(phone)
                        equipment_data['email'].append(email)
                        equipment_data['fantasy_name'].append(fantasy_name)
                        equipment_data['cep'].append(cep)
                        equipment_data['street_name'].append(street_name)
                        equipment_data['address_number'].append(address_number)
                        equipment_data['address_complement'].append(address_complement)
                        equipment_data['neighborhood_name'].append(neighborhood_name)
                        equipment_data['city_name'].append(city_name)
                        equipment_data['state_name'].append(state_name)

                    print(f'current_data: {equipment_data}')

                    break  # Sai do `while` se tudo der certo

                except StaleElementReferenceException:
                    retries += 1
                    time.sleep(1)  # Tenta novamente após um pequeno delay

            change_window('Pesquisa')
            time.sleep(1)

        except Exception as e:
            print(f"Erro ao processar cliente {i+1}: {e}")
            break

    # Verifica se o botão "Próximo" está desativado
    try:
        next_button_li = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "tabelaClientes_next"))
        )
        
        if 'disabled' in next_button_li.get_attribute('class'):
            print("Fim da paginação. Encerrando.")
            break
            
        next_button = next_button_li.find_element(By.TAG_NAME, "a")
        page += 1
        next_button.click()

    except Exception as e:
        print(f"Erro ao clicar em 'Próximo': {e}")
        break  # Sai do loop se houver erro na paginação

1/10 da pag. 1
Aba 'Processando...' desapareceu. Continuando...
Erro ao processar cliente 1: 'fantasy_name'
Erro ao clicar em 'Próximo': Message: element not interactable
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF64E936EE5+28773]
	(No symbol) [0x00007FF64E8A25D0]
	(No symbol) [0x00007FF64E738DDC]
	(No symbol) [0x00007FF64E7906B4]
	(No symbol) [0x00007FF64E782284]
	(No symbol) [0x00007FF64E7B724A]
	(No symbol) [0x00007FF64E781B36]
	(No symbol) [0x00007FF64E7B7460]
	(No symbol) [0x00007FF64E7DF6F3]
	(No symbol) [0x00007FF64E7B7023]
	(No symbol) [0x00007FF64E77FF5E]
	(No symbol) [0x00007FF64E7811E3]
	GetHandleVerifier [0x00007FF64EC8422D+3490733]
	GetHandleVerifier [0x00007FF64EC9BA13+3586963]
	GetHandleVerifier [0x00007FF64EC9144D+3544525]
	GetHandleVerifier [0x00007FF64E9FC9AA+838442]
	(No symbol) [0x00007FF64E8AD01F]
	(No symbol) [0x00007FF64E8A95E4]
	(No symbol) [0x00007FF64E8A9786]
	(No symbol) [0x00007FF64E898CB9]
	BaseThreadInitThunk [0x00007FF

In [91]:
df = pd.DataFrame(equipment_data)
df.to_csv('equipment_data.csv')

In [93]:
pd.DataFrame(equipment_data).to_excel('equipment_data.xlsx')